In [0]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

# install twitter scraper - use !pip because it isn't included with Google Colab / Jupyter by default
!pip install -q twint

#imports
import twint
import logging 
import pandas as pd
import lxml.etree
import datetime as dt
from multiprocessing.pool import ThreadPool
import time
from datetime import datetime  
from datetime import timedelta 
from pytz import timezone
import os


print('\n\nFinished importing things and setting up environment!  At ' + str(datetime.now(timezone('EST'))))

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
     |████████████████████████████████| 1.2MB 7.6MB/s 
     |████████████████████████████████| 245kB 64.2MB/s 
     |████████████████████████████████| 92kB 11.3MB/s 
     |████████████████████████████████| 256kB 61.6MB/s 
     |████████████████████████████████| 153kB 68.3MB/s 
     |████████████████████████████████| 235kB 55.8MB/s 


Finished importing things and setting up environment!  At 2020-03-11 22:02:58.417898-05:00


filename is:  drive/My Drive/Documents/SCHOOL/Watson CI/Parks Project - Watson CI/Twint/Twint Multi Year Pulltwint_2018-01-01_to_2020-03-01.csv


In [0]:
# set some parameters and file names
# path = "drive/My Drive/Twitter Scraper Data/"
path = "drive/My Drive/Documents/SCHOOL/Watson CI/Parks Project - Watson CI/Twint/Twint Multi Year Pull/"
# start_date = "2018-01-01"
# end_date = "2020-03-01"  #<-- overall start and end dates, lets do month by month though
start_date = "2017-12-19"
end_date = "2017-12-20"
filename = path + "twint_" + start_date + "_to_" + end_date + ".csv"  #The file that Twint will create with the tweets

# FIXME comment out limit when ready 
# limit = 100  #Use this to just gather 100 tweets then stop
print("filename is: ",filename)

#configuring the search 
# We used the Advanced Search menu on Twitter's website to learn how to exclude/include/AND/OR within their search. 
c = twint.Config()
c.Search = '(jtnp OR joshua_tree OR joshuatree OR "joshua tree") -U2 -album -achtung -"Unforgettable Fire"'
c.Output = filename
c.Lang = "en"
c.Location = True
c.Since = start_date
c.Until = end_date
c.Store_csv = True
c.Show_hashtags = True
c.Filter_retweets = True
# c.Limit = limit  #comment this out I guess
c.Count = True
c.Stats = True
c.Hide_output = True
#c.Min_likes = 5 

filename is:  drive/My Drive/Documents/SCHOOL/Watson CI/Parks Project - Watson CI/Twint/Twint Multi Year Pull/twint_2017-12-19_to_2017-12-20.csv


In [0]:
# Run Once
start_time = time.time()
twint.run.Search(c)
duration = time.time() - start_time
print("Finished gathering tweets between " + start_date + " and " + end_date + ".  Took " + str(duration) + " seconds")

[+] Finished: Successfully collected 170 Tweets.
Finished gathering tweets between 2017-12-19 and 2017-12-20.  Took 108.42078733444214 seconds


# Batch Processing of Tweet Gathering

In [0]:
# Note - it takes like 6 hours to do this.  Seems like it can do 1.5 tweets per second, so if you have 200,000 tweets it will be almost 40 hours. 
# Run for a bunch of months
path = "drive/My Drive/Documents/SCHOOL/Watson CI/Parks Project - Watson CI/Twint/Twint Multi Year Pull/"

# I split up by year because Google Colab runtime expires after 12 hours 
stamps_2017 = ["2017-01-01", "2017-02-01", "2017-03-01", "2017-04-01", "2017-05-01", "2017-06-01", "2017-07-01", "2017-08-01", "2017-09-01", "2017-10-01", "2017-11-01", "2017-12-01", "2018-01-01"]
stamps_2018 = ["2018-01-01", "2018-02-01", "2018-03-01", "2018-04-01", "2018-05-01", "2018-06-01", "2018-07-01", "2018-08-01", "2018-09-01", "2018-10-01", "2018-11-01", "2018-12-01", "2019-01-01"]
stamps_2019 = ["2019-01-01", "2019-02-01", "2019-03-01", "2019-04-01", "2019-05-01", "2019-06-01", "2019-07-01", "2019-08-01", "2019-09-01", "2019-10-01", "2019-11-01", "2019-12-01", "2020-01-01"]
stamps_2020 = ["2020-01-01", "2020-02-01", "2020-03-01"]

year = 2020
path = path + str(year) + "/"

h = 0
tz = timezone('EST')
duration = 27.6 # minutes, estimated time to pull a month of data.  
while h < (len(stamps_2020) - 1):
  start_time = time.time()
  start_date = stamps_2020[h]
  end_date = stamps_2020[h+1]

  #New file path - VERY IMPORTANT don't overwrite the nice file we just made
  filename = path + "twint_" + start_date + "_to_" + end_date + ".csv"
  print("\nGathering data for " + start_date + " to " + end_date + "...")
  print("Started at " + str(datetime.now(tz)) + ",  should end around " + str(datetime.now(tz) + timedelta(minutes=duration))  )
  

  #Do the thing here
  #configuring the search 
  c = twint.Config()
  c.Search = '(jtnp OR joshua_tree OR joshuatree OR "joshua tree") -U2 -album -achtung -"Unforgettable Fire"'
  c.Output = filename  # this is really all we need, I felt better having everything here though. 
  c.Lang = "en"
  c.Location = True
  c.Since = start_date
  c.Until = end_date
  c.Store_csv = True
  c.Show_hashtags = True
  c.Filter_retweets = True
  # c.Limit = 20  #comment this out if you don't want to limit yourself
  c.Count = True
  c.Stats = True
  c.Hide_output = True

  twint.run.Search(c)

  #How long did it take?
  duration = (time.time() - start_time) / 60
  print("\nFinished gathering tweets between " + start_date + " and " + end_date + ".  Took " + str(duration) + " minutes")

  h += 1




Gathering data for 2020-01-01 to 2020-02-01...
Started at 2020-03-05 22:43:00.235658-05:00,  should end around 2020-03-05 23:10:36.235683-05:00


CRITICAL:root:twint.get:Tweet:Response payload is not completed


[+] Finished: Successfully collected 4218 Tweets.

Finished gathering tweets between 2020-01-01 and 2020-02-01.  Took 33.54111537535985 minutes

Gathering data for 2020-02-01 to 2020-03-01...
Started at 2020-03-05 23:16:32.702676-05:00,  should end around 2020-03-05 23:50:05.169624-05:00
[+] Finished: Successfully collected 3900 Tweets.

Finished gathering tweets between 2020-02-01 and 2020-03-01.  Took 29.609973029295602 minutes


In [0]:
# view output csv with pandas
# TODO fixe this to include column names 
jtnp_tweets = pd.read_csv(filename)

jtnp_tweets.columns = ['id','conversation_id','created_at',
                     'date','time','timezone','user_id','username','name','place','tweet','mentions','urls','photos','replies_count','retweets_count','likes_count','hashtags','cashtags','link','retweet','quote_url','video','near','geo','source','user_rt_id','user_rt','retweet_id','reply_to','retweet_date','translate','trans_src','trans_dest']

jtnp_tweets

In [0]:
# print a few of them
for txt in jtnp_tweets.iloc[:,10]:
  print(txt)


# Combine CSVs

In [0]:
# Join all CSVs in a folder into one 
# Note - DO NOT delete the headers.  It uses them to concatenate the files and actually doens't repeat them.  Pretty nifty. 
from os import chdir
import glob

year = 2021
year_str = str(year)

folder_with_files_to_join = "drive/My Drive/Documents/SCHOOL/Watson CI/Parks Project - Watson CI/Twint/Twint Multi Year Pull/"+year_str+"/"
path = folder_with_files_to_join

all_files = glob.glob(os.path.join(path, "*.csv"))
print(all_files)
df_from_each_file = (pd.read_csv(f) for f in all_files)
concatenated_df   = pd.concat(df_from_each_file, ignore_index=True, sort=None)

print("Length before cleaning:  " + str(len(concatenated_df)))

#Simple minimal cleaning
#Drop Duplicates - there shouldn't be any, but who knows
concatenated_df.drop_duplicates()
concatenated_df.dropna(how='all', inplace=True)  # Drop rows with NaN or missing value in all columns
concatenated_df.dropna(how='all', subset=['id', 'conversation_id'], inplace=True)


print("Length after cleaning:  " + str(len(concatenated_df)))



file_out = "drive/My Drive/Documents/SCHOOL/Watson CI/Parks Project - Watson CI/Twint/Twint Multi Year Pull/"+year_str+" ALL Combined.csv"
concatenated_df.to_csv(file_out, index=False)
print("done")


# Instagram picture pulling

In [0]:
from itertools import dropwhile, takewhile
import glob
import json
import shutil
import os

!pip install instaloader
import instaloader
print("Imported/Installed Instaloader sucessfully")

In [0]:
# I found this library and thought it was interesting for future work.  
# We didn't use it on our project but maybe the next group will.  Easy way
# to pull a bunch of pictures about a subject.  

# FUNCTION DEFINITIONS

L = instaloader.Instaloader(download_pictures=True,
                            download_video_thumbnails=False,
                            download_videos=False,
                            download_geotags=True,
                            download_comments=False,
                            save_metadata=True,
                            compress_json=False
                            )

# two arguments: keyword and number of files
def get_it(keyword,numbers_of_files):
    while numbers_of_files > 0:
        for post in L.get_hashtag_posts(keyword):
            # post is an instance of instaloader.Post
            L.download_post(post, target='#'+keyword)
            numbers_of_files-=1
            # print(numbers_of_files)
            if numbers_of_files ==0:
                break
        print("loop ended for " + keyword)


        
# using Instagram's image recognition: it is written on json file, with key of "__typename"
def discern_image_file(keyword):
    # read file list that is previously downloaded through get_it
    read_files_list = glob.glob("#"+keyword+"/*.json")
    print(read_files_list)
    write_files_list = []
    # open every json file, check whether it is streetart image
    # It seems like streetart image is either classified as "GraphSidecar" or "GraphImage" through Instagram image recognition
    for json_file_name in read_files_list:
        # open json file
        with open(json_file_name,'r',encoding="UTF8") as data_file:
            # read json file
            data = json.load(data_file)
            file_name = json_file_name[:-5]
            # print(data)  # fair warning this prints a LOT of information
            #Pull some data out of it:
            this_caption = data["node"]["edge_media_to_caption"]["edges"][0]["node"]["text"]
            this_url = data["node"]["display_resources"][0]["src"]
            this_caption = this_caption.replace('\r.', '').replace('\n.', '').replace('\r-', '').replace('\n-', '').replace('\r', ' ').replace('\n', ' ')  #Get everything onto one line
            this_caption = this_caption.replace(' .','').replace('•','') #People use a character on each line 
            print(this_caption + "      " + this_url)

            if data["node"]["__typename"] == "GraphSidecar":
                # print(file_name + " is GraphSidecar typename")
                write_files_list.append(file_name)
            elif data["node"]["__typename"] == "GraphImage":  #all of ours seem to be this
                # print(file_name + " is GraphImage typename")
                write_files_list.append(file_name)
            else:
                print(file_name + " is not streetarts image")
                pass
    # print(write_files_list)
    return write_files_list


def sort_and_move_image_file(keyword):   
    # change the source directory to your project directory
    # source = '/path/to/source_folder'
    source = ""  # For colab use just leave it blank and it magically works
    dest   = "drive/My Drive/Documents/SCHOOL/Artifical Neural Networks 8720/Takehome 2 files/" + keyword + "_sorted/"

    # make directory of representative pictures only
    os.mkdir(dest)

         
    all_image_file_list = glob.glob("#"+keyword+"/*.jpg")
    write_files_list = discern_image_file(keyword)
    print(all_image_file_list)
    print(write_files_list)
    for item in write_files_list:
        matching_list = [s for s in all_image_file_list if item in s]
        print(matching_list)
        for matching_item in matching_list:
            print(matching_item)
            shutil.move(source+matching_item, dest)

print("Set up Functions sucessfully")


# posts = L.get_hashtag_posts('jtnp')
# or
# posts = instaloader.Profile.from_username(L.context, PROFILE).get_posts()
# SINCE = datetime(2020, 1, 1)
# UNTIL = datetime(2020, 1, 3)
# for post in takewhile(lambda p: p.date > UNTIL, dropwhile(lambda p: p.date > SINCE, posts)):
#     print(post.date)
#     L.download_post(post, '#jtnp')

Set up Functions sucessfully


In [0]:
my_keyword = "joshuatreenationalpark"
# sample function just to get the pics
# get_it(my_keyword,2)


# discern image files only
discern_image_file(my_keyword)

# discern, sort, and move image files to separate directory
# sort_and_move_image_file(my_keyword)

['#joshuatreenationalpark/2020-03-11_02-15-52_UTC.json', '#joshuatreenationalpark/2020-03-11_02-44-10_UTC.json', '#joshuatreenationalpark/2020-03-11_02-21-02_UTC.json', '#joshuatreenationalpark/2020-03-11_02-28-19_UTC.json', '#joshuatreenationalpark/2020-03-11_01-39-54_UTC.json', '#joshuatreenationalpark/2020-03-11_02-32-09_UTC.json', '#joshuatreenationalpark/2020-03-11_02-11-42_UTC.json', '#joshuatreenationalpark/2020-03-11_01-36-56_UTC.json', '#joshuatreenationalpark/2020-03-11_01-40-12_UTC.json', '#joshuatreenationalpark/2020-03-11_01-36-46_UTC.json', '#joshuatreenationalpark/2020-03-11_02-05-29_UTC.json', '#joshuatreenationalpark/2020-03-11_01-38-39_UTC.json', '#joshuatreenationalpark/2020-03-11_01-58-18_UTC.json', '#joshuatreenationalpark/2020-03-11_02-42-46_UTC.json', '#joshuatreenationalpark/2020-03-11_02-06-00_UTC.json', '#joshuatreenationalpark/2020-03-11_02-08-44_UTC.json', '#joshuatreenationalpark/2020-03-11_01-51-58_UTC.json', '#joshuatreenationalpark/2020-03-11_01-47-41_UT

['#joshuatreenationalpark/2020-03-11_02-15-52_UTC',
 '#joshuatreenationalpark/2020-03-11_02-44-10_UTC',
 '#joshuatreenationalpark/2020-03-11_02-21-02_UTC',
 '#joshuatreenationalpark/2020-03-11_02-28-19_UTC',
 '#joshuatreenationalpark/2020-03-11_01-39-54_UTC',
 '#joshuatreenationalpark/2020-03-11_02-11-42_UTC',
 '#joshuatreenationalpark/2020-03-11_01-36-56_UTC',
 '#joshuatreenationalpark/2020-03-11_01-40-12_UTC',
 '#joshuatreenationalpark/2020-03-11_01-36-46_UTC',
 '#joshuatreenationalpark/2020-03-11_02-05-29_UTC',
 '#joshuatreenationalpark/2020-03-11_01-38-39_UTC',
 '#joshuatreenationalpark/2020-03-11_01-58-18_UTC',
 '#joshuatreenationalpark/2020-03-11_02-42-46_UTC',
 '#joshuatreenationalpark/2020-03-11_02-06-00_UTC',
 '#joshuatreenationalpark/2020-03-11_02-08-44_UTC',
 '#joshuatreenationalpark/2020-03-11_01-51-58_UTC',
 '#joshuatreenationalpark/2020-03-11_01-47-41_UTC',
 '#joshuatreenationalpark/2020-03-11_01-41-01_UTC',
 '#joshuatreenationalpark/2020-03-11_02-26-22_UTC',
 '#joshuatre